In [ ]:
# Grid

In [1]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import time
import hmac
import hashlib
import base64
import json
from datetime import datetime

In [3]:
# OKX API Credentials
api_key = '43069784-de20-4127-bc0c-c72f233b93ac'
secret_key = '87AA1870EAA99B8857376D66379DDB31'
passphrase = 'Rachel0622!'

In [4]:
# Function to Create the Signature Required by OKX API
def generate_signature(timestamp, method, request_path, body, secret_key):
    message = timestamp + method + request_path + body
    mac = hmac.new(bytes(secret_key, 'utf-8'), bytes(message, 'utf-8'), digestmod=hashlib.sha256)
    d = mac.digest()
    return base64.b64encode(d).decode('utf-8')

In [5]:
# Function to Fetch the Current Market Price from OKX
def fetch_market_price(symbol):
    base_url = 'https://www.okx.com'
    endpoint = f'/api/v5/market/ticker?instId={symbol}'
    url = base_url + endpoint

    timestamp = str(time.time())
    method = 'GET'
    request_path = endpoint
    body = ''
    signature = generate_signature(timestamp, method, request_path, body, secret_key)

    headers = {
        'OK-ACCESS-KEY': api_key,
        'OK-ACCESS-SIGN': signature,
        'OK-ACCESS-TIMESTAMP': timestamp,
        'OK-ACCESS-PASSPHRASE': passphrase,
        'Content-Type': 'application/json'
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    if 'data' in data:
        return float(data['data'][0]['last'])
    else:
        raise Exception(f"Error fetching data: {data}")

In [ ]:
### FOR BINANCE US API ###

# Binance US API Credentials
api_key = 'YOUR_API_KEY'
secret_key = 'YOUR_SECRET_KEY'

# Function to Create the Signature of Binance US API
def generate_signature(query_string, secret_key):
    return hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

#Function to Fetch the Current Market Price from Binance US
def fetch_market_price(symbol):
    base_url = 'https://api.binance.us'
    endpoint = f'/api/v3/ticker/price?symbol={symbol}'
    url = base_url + endpoint

    response = requests.get(url)
    data = response.json()
    if 'price' in data:
        return float(data['price'])
    else:
        raise Exception(f"Error fetching data: {data}")

In [6]:
# Example usage
symbol = 'BTC-USDT'
try:
    current_price = fetch_market_price(symbol)
    print(f"Current price of {symbol}: ${current_price}")
except Exception as e:
    print(e)

Current price of BTC-USDT: $68521.1


In [7]:
import random

def create_grid(current_price, lower_bound, upper_bound, num_levels):
    interval = (upper_bound - lower_bound) / num_levels
    buy_orders = []
    sell_orders = []

    for i in range(num_levels):
        buy_price = lower_bound + i * interval
        sell_price = upper_bound - i * interval
        if buy_price < current_price:
            buy_orders.append(buy_price)
        if sell_price > current_price:
            sell_orders.append(sell_price)

    return buy_orders, sell_orders

def simulate_trading(buy_orders, sell_orders, initial_balance, initial_coins, price_fluctuations):
    balance = initial_balance
    coins = initial_coins

    for price in price_fluctuations:
        # Execute buy orders
        for buy_price in buy_orders:
            if price <= buy_price and balance >= buy_price:
                coins += 1
                balance -= buy_price
                print(f"Bought 1 coin at ${buy_price:.2f}")

        # Execute sell orders
        for sell_price in sell_orders:
            if price >= sell_price and coins > 0:
                coins -= 1
                balance += sell_price
                print(f"Sold 1 coin at ${sell_price:.2f}")

    final_balance = balance + coins * price_fluctuations[-1]
    print(f"Final balance: ${final_balance:.2f}")
    return final_balance

In [8]:
# Example usage
symbol = 'BTC-USDT'
try:
    current_price = fetch_market_price(symbol)
    print(f"Current price of {symbol}: ${current_price}")
    
    lower_bound = 30000
    upper_bound = 40000
    num_levels = 10

    buy_orders, sell_orders = create_grid(current_price, lower_bound, upper_bound, num_levels)
    print("Buy Orders:", buy_orders)
    print("Sell Orders:", sell_orders)

    initial_balance = 10000
    initial_coins = 0
    price_fluctuations = [current_price + random.uniform(-1000, 1000) for _ in range(30)]

    simulate_trading(buy_orders, sell_orders, initial_balance, initial_coins, price_fluctuations)
except Exception as e:
    print(e)

Current price of BTC-USDT: $68508.1
Buy Orders: [30000.0, 31000.0, 32000.0, 33000.0, 34000.0, 35000.0, 36000.0, 37000.0, 38000.0, 39000.0]
Sell Orders: []
Final balance: $10000.00


In [ ]:
# Place Real Orders on OKX

#ef place_order(symbol, side, size, price):
#    base_url = 'https://www.okx.com'
#    endpoint = '/api/v5/trade/order'
#    url = base_url + endpoint

#   timestamp = str(time.time())
#   method = 'POST'
#   request_path = endpoint
#   body = {
#       "instId": symbol,
#       "tdMode": "cash",
#       "side": side,
#       "ordType": "limit",
#       "sz": str(size),
#       "px": str(price)
#   }
#   body_str = json.dumps(body)
#   signature = generate_signature(timestamp, method, request_path, body_str, secret_key)

#   headers = {
#       'OK-ACCESS-KEY': api_key,
#       'OK-ACCESS-SIGN': signature,
#       'OK-ACCESS-TIMESTAMP': timestamp,
#       'OK-ACCESS-PASSPHRASE': passphrase,
#       'Content-Type': 'application/json'
#   }

#   response = requests.post(url, headers=headers, data=body_str)
#   data = response.json()
#   if 'data' in data:
#       return data['data']
#   else:
#       raise Exception(f"Error placing order: {data}")

# Example usage

#symbol = 'BTC-USDT'
#side = 'buy'
#size = 0.001  # size in BTC
#price = 30000  # limit price
#try:
#   order = place_order(symbol, side, size, price)
#   print(f"Order placed: {order}")
#except Exception as e:
#   print(e)

In [ ]:
#DCA Trading

In [9]:
# OKX API credentials (Creating Credentials remains the same)
api_key = '43069784-de20-4127-bc0c-c72f233b93ac'
secret_key = '87AA1870EAA99B8857376D66379DDB31'
passphrase = 'Rachel0622!'

In [10]:
# Function to create the signature required by OKX API
def generate_signature(timestamp, method, request_path, body, secret_key):
    message = timestamp + method + request_path + body
    mac = hmac.new(bytes(secret_key, 'utf-8'), bytes(message, 'utf-8'), digestmod=hashlib.sha256)
    d = mac.digest()
    return base64.b64encode(d).decode('utf-8')

In [11]:
# Function to fetch the current market price from OKX
def fetch_market_price(symbol):
    base_url = 'https://www.okx.com'
    endpoint = f'/api/v5/market/ticker?instId={symbol}'
    url = base_url + endpoint

    timestamp = str(time.time())
    method = 'GET'
    request_path = endpoint
    body = ''
    signature = generate_signature(timestamp, method, request_path, body, secret_key)

    headers = {
        'OK-ACCESS-KEY': api_key,
        'OK-ACCESS-SIGN': signature,
        'OK-ACCESS-TIMESTAMP': timestamp,
        'OK-ACCESS-PASSPHRASE': passphrase,
        'Content-Type': 'application/json'
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    if 'data' in data:
        return float(data['data'][0]['last'])
    else:
        raise Exception(f"Error fetching data: {data}")

In [12]:
# Example usage
symbol = 'BTC-USDT'
try:
    current_price = fetch_market_price(symbol)
    print(f"Current price of {symbol}: ${current_price}")
except Exception as e:
    print(e)

Current price of BTC-USDT: $68500.6


In [13]:
# Implement the DCA Trading
def place_order(symbol, side, size, price):
    base_url = 'https://www.okx.com'
    endpoint = '/api/v5/trade/order'
    url = base_url + endpoint

    timestamp = str(time.time())
    method = 'POST'
    request_path = endpoint
    body = {
        "instId": symbol,
        "tdMode": "cash",
        "side": side,
        "ordType": "limit",
        "sz": str(size),
        "px": str(price)
    }
    body_str = json.dumps(body)
    signature = generate_signature(timestamp, method, request_path, body_str, secret_key)

    headers = {
        'OK-ACCESS-KEY': api_key,
        'OK-ACCESS-SIGN': signature,
        'OK-ACCESS-TIMESTAMP': timestamp,
        'OK-ACCESS-PASSPHRASE': passphrase,
        'Content-Type': 'application/json'
    }

    response = requests.post(url, headers=headers, data=body_str)
    data = response.json()
    if 'data' in data:
        return data['data']
    else:
        raise Exception(f"Error placing order: {data}")

def dca_investment(symbol, interval, amount, duration):
    for _ in range(duration):
        current_price = fetch_market_price(symbol)
        if current_price:
            size = amount / current_price
            try:
                order = place_order(symbol, 'buy', size, current_price)
                print(f"Placed DCA order: {order}")
            except Exception as e:
                print(f"Error placing order: {e}")
        else:
            print("Failed to fetch the current price.")
        time.sleep(interval)

In [ ]:
# Example usage
symbol = 'BTC-USDT'
interval = 86400  # 24 hours in seconds
amount = 100  # $100 investment per interval
duration = 7  # 7 days

dca_investment(symbol, interval, amount, duration)

Error placing order: Error placing order: {'msg': 'Invalid OK-ACCESS-TIMESTAMP', 'code': '50112'}


In [ ]:
#Scheduling DCA Investment

In [ ]:
#import schedule

#def dca_investment_schedule():
#   symbol = 'BTC-USDT'
#   amount = 100  # $100 investment per interval
#   try:
#       current_price = fetch_market_price(symbol)
#       if current_price:
#           size = amount / current_price
#           order = place_order(symbol, 'buy', size, current_price)
#           print(f"Placed DCA order: {order}")
#       else:
#           print("Failed to fetch the current price.")
#   except Exception as e:
#       print(f"Error placing order: {e}")

# Schedule the DCA investment every 24 hours
#schedule.every().day.at("09:00").do(dca_investment_schedule)

#while True:
#   schedule.run_pending()
#   time.sleep(1)